<a href="https://colab.research.google.com/github/vaguiar/capstone-project-2019/blob/master/src/explore/Normalize_2017_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Normalizing the data
The Data values in the provisional dataset are reproted in rolling 1 year windows. ie. in Jan 2018, the number reported is the sum of eveything from Feb 2107 - Jan 2018. So in order to get only the Jan 2018 numbers, we needed to extract the last year's CDC WONDER data values for all states i.e. Jan '17 - Dec '17 for all states first. Next we took difference between every consecutive rolling year value reported and added the prev year's value to it.

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cdc_link = '/content/drive/My Drive/Capstone Project 2019/data/cdc-wonder/cdc-provisional-normalized/cdc_opioids_state_normalized.csv'


In [0]:
import pandas as pd

cdc = pd.DataFrame(pd.read_csv(cdc_link))

In [5]:
month_map = {
  'Jan.':1,
  'Feb.':2,
  'Mar.':3,
  'Apr.':4,
  'May' :5, 
  'Jun.':6,
  'Jul.':7,
  'Aug.':8,
  'Sep.':9,
  'Oct.':10,
  'Nov.':11,
  'Dec.':12
}

for i in month_map.keys():
  print(i)

Jan.
Feb.
Mar.
Apr.
May
Jun.
Jul.
Aug.
Sep.
Oct.
Nov.
Dec.


In [6]:
cdc_2017 = cdc[cdc['Year'] == 2017]

len(cdc_2017)/12

47.416666666666664

In [7]:
cdc_2017

,State,State Code,Month,Year,Monthly Code,Deaths
36,Alabama,1,Jan.,2017,1,91
37,Alabama,1,Feb.,2017,2,73
38,Alabama,1,Mar.,2017,3,92
39,Alabama,1,Apr.,2017,4,75
40,Alabama,1,May,2017,5,83
...,...,...,...,...,...,...
2233,Wisconsin,55,Oct.,2017,10,109
2234,Wisconsin,55,Nov.,2017,11,81
2235,Wisconsin,55,Dec.,2017,12,97
2251,Wyoming,56,Oct.,2017,10,13


In [38]:
import copy

# insert rows for missing months with count = 0. Do only for 2017
cdc_copy = pd.DataFrame(columns=['State', 'State Code', 'Month', 'Year', 'Monthly Code', 'Deaths'])

# # Uncomment to Test with Alaska
# al_cdc = cdc_2017[cdc_2017['State'] == 'Alaska']
# al_cdc = al_cdc.reset_index(drop=True, inplace=False)

idx = 0
while idx < len(cdc_2017.index):
  curr = cdc_2017.iloc[idx]
  print(idx, curr['State'], curr['Month'])

  for mth in month_map.keys():
    if str(mth) != str(curr['Month']):
      print(f"missing month {mth}")
      print(f"insert {mth}")

      # insert missing row
      row = copy.deepcopy(curr).to_dict()
      row['Month'] = mth                 # insert the missing month
      row['Monthly Code'] = month_map[mth] # give it the id
      row['Deaths'] = 0                  # setting death count to 0
      print(f"insert row {row}")
      cdc_copy = cdc_copy.append(row, ignore_index=True, sort=False)
    else:
      print(f"matching {mth}")
      cdc_copy = cdc_copy.append(curr)
      if str(mth) == str('Dec.'):
        idx+=1
        break
      
      idx+= 1

      if idx < len(cdc_2017.index): # don't go past the last value in the dataset
        curr = cdc_2017.iloc[idx]
        print(idx, curr['State'], curr['Month'])

print(cdc_copy)


0 Alabama Jan.
matching Jan.
1 Alabama Feb.
matching Feb.
2 Alabama Mar.
matching Mar.
3 Alabama Apr.
matching Apr.
4 Alabama May
matching May
5 Alabama Jun.
matching Jun.
6 Alabama Jul.
matching Jul.
7 Alabama Aug.
matching Aug.
8 Alabama Sep.
matching Sep.
9 Alabama Oct.
matching Oct.
10 Alabama Nov.
matching Nov.
11 Alabama Dec.
matching Dec.
12 Alaska Jan.
matching Jan.
13 Alaska Mar.
missing month Feb.
insert Feb.
insert row {'State': 'Alaska', 'State Code': 2, 'Month': 'Feb.', 'Year': 2017, 'Monthly Code': 2, 'Deaths': 0}
matching Mar.
14 Alaska May
missing month Apr.
insert Apr.
insert row {'State': 'Alaska', 'State Code': 2, 'Month': 'Apr.', 'Year': 2017, 'Monthly Code': 4, 'Deaths': 0}
matching May
15 Alaska Jul.
missing month Jun.
insert Jun.
insert row {'State': 'Alaska', 'State Code': 2, 'Month': 'Jun.', 'Year': 2017, 'Monthly Code': 6, 'Deaths': 0}
matching Jul.
16 Alaska Aug.
matching Aug.
17 Alaska Sep.
matching Sep.
18 Alaska Nov.
missing month Oct.
insert Oct.
insert r

In [41]:
print("We got all months for all ", len(cdc_copy)/12, " states") 

We got all months for all  51.0  states


In [42]:
cdc_copy

,State,State Code,Month,Year,Monthly Code,Deaths
0,Alabama,1,Jan.,2017,1,91
1,Alabama,1,Feb.,2017,2,73
2,Alabama,1,Mar.,2017,3,92
3,Alabama,1,Apr.,2017,4,75
4,Alabama,1,May,2017,5,83
...,...,...,...,...,...,...
607,Wyoming,56,Aug.,2017,8,0
608,Wyoming,56,Sep.,2017,9,0
609,Wyoming,56,Oct.,2017,10,13
610,Wyoming,56,Nov.,2017,11,14


### Upload file to gdrive

**Don't rewrite this filename**

In [0]:
# cdc_copy.to_csv('cdc_opioid_state_w_missing.csv')
# !cp cdc_opioid_state_w_missing.csv drive/My\ Drive/Capstone\ Project\ 2019/data/cdc-wonder/cdc-provisional-normalized/